# Assignment 1

## Author: Saurabh Sharma

In [ ]:
# All imports

import os
from pathlib import Path
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Ensure outputs directory
out_dir = Path('outputs')
out_dir.mkdir(exist_ok=True)

csv_path = Path('data') / 'chicago_violations.csv'
print('Reading:', csv_path.resolve())

In [ ]:

# Read CSV with pandas, try to infer dates later
df = pd.read_csv(csv_path, low_memory=False)
print('\nInitial shape:', df.shape)


In [ ]:
# 1. Describe dataset
print('\nDataframe info:')
df.info()

print('\nFirst 5 rows:')
print(df.head())

# 1. How many rows and columns
n_rows, n_cols = df.shape
print(f"\nNumber of rows: {n_rows}\nNumber of columns: {n_cols}")

In [ ]:
for i, c in enumerate(df.columns, start=1):
    print(f"{i:2}. {c}")

In [ ]:
# Convert to numeric and show the top 5 ward values by frequency, then choose one at random from those
wards_present = pd.to_numeric(df['WARD'], errors='coerce')
# coerce = convert invalid to 'NaN'

# value_counts on non-null wards
top5 = pd.Series(data=wards_present.dropna(), name="wards").value_counts().nlargest(5)
print('\nTop 5 WARD values (by count):')
print(top5)

# Choose randomly from the top 5 most common ward values
top5_values = top5.index.astype(int).tolist()
if len(top5_values) == 0:
    raise ValueError('No numeric wards detected')

# chosen_ward = int(random.choice(top5_values))
# print('Chosen (random) ward (from top 5):', chosen_ward)
chosen_ward = 20
print('Chosen (random) ward (from top 5):', chosen_ward)

In [ ]:
# Assign the numeric ward column from the already-computed series
df_ward = df[wards_present == chosen_ward].copy()
print('\nFiltered shape (chosen ward):', df_ward.shape)

In [ ]:
# 1. How many records now
print('Records in chosen ward:', len(df_ward))
df_ward.head(6)

In [ ]:
# 2. Identify at least three interesting facts about your ward
# Part A: Data-driven facts from df_ward (the filtered dataset for this ward)
print("\n" + "="*80)
print(f"WARD {chosen_ward}: FACTS AND INFORMATION")
print("="*80)

most_common_street = df_ward['STREET NAME'].value_counts()
print(f" 1. Most common street : {most_common_street.index[0]}")
print(" Intersting fact: Martin Luther King Jr. was born Michael King Jr. but changed his name in 1934 to honor the Protestant reformer Martin Luther.")

most_common_street_type = df_ward['STREET TYPE'].value_counts()
print(f" 2. Most common street type : {most_common_street_type.index[0]}")
print(" Interesting fact: Avenues, in contrast to streets, usually run from north to south and have medians.")

# Get most common violation code
most_common_violation_code = df_ward['VIOLATION CODE'].value_counts().index[0]
least_common_violation_code = df_ward['VIOLATION CODE'].value_counts().index[-1]
# Get the description for that violation code
# Filter rows where VIOLATION CODE matches the most common code
violation_desc_for_code = df_ward[df_ward['VIOLATION CODE'] == most_common_violation_code]['VIOLATION DESCRIPTION'].iloc[0]

print(f" 3. Most common violation code : {most_common_violation_code}")
print(f"    Description: {violation_desc_for_code}")
print("""
        The top 10 most common violation codes for the past year are:
          CN190019: Arrange premise inspection
          EV1110: Maintain or repair elect eleva
          CN061014: Repair exterior wall
          EV1111: Maintain or repair hydro eleva
          CN070024: Repair porch system
          CN138106: Stop/remove nuisance
          CN101015: Repair interior walls/ceiling
          NC2011: Plans & permits req - contrctr
          CN070014: Repair exterior stair
          CN197019: Install smoke detectors
      """)
print(f" 4. Least common violation code : {least_common_violation_code}")


In [ ]:
# DEAL WITH NAs
print(f'\nCounting NA values in ward={chosen_ward} dataframe...')
na_counts = df_ward.isna().sum()
na_counts

In [ ]:
print('Total number of NA values in original dataframe:', na_counts.sum())

In [ ]:
# Percentage of complete cases
print('A complete case is a row with no missing (NA) values in any column.')
complete_cases = df_ward.dropna()
pct_complete = 100 * len(complete_cases) / len(df_ward) if len(df_ward)>0 else np.nan
print(f"\nPercent of rows that are complete cases: {pct_complete:.2f}%")


In [ ]:
# Convert blank strings to NA
blank_count_before = (df_ward == '').sum().sum()
print('Blank-string cells before conversion:', blank_count_before)

df_ward = df_ward.replace(r'^\s*$', np.nan, regex=True)

blank_count_after = (df_ward == '').sum().sum()
print('Blank-string cells after conversion:', blank_count_after)

In [ ]:
# Generate per-variable missing table
missing_table = pd.DataFrame({'missing_count': df_ward.isna().sum()})
missing_table['missing_percent'] = 100 * missing_table['missing_count'] / len(df_ward)
missing_table = missing_table.sort_values('missing_count', ascending=False)
missing_table.to_csv(out_dir / 'missing_table.csv')
print('\nMissing values table saved to outputs/missing_table.csv')

In [ ]:
# Format and print missing_percent as .2f%
missing_table_display = missing_table.copy()
missing_table_display['missing_percent'] = missing_table_display['missing_percent'].apply(lambda x: f"{x:.2f}%")
missing_table_display

In [ ]:
# Which variables contain date values? Check dtype first
print('\nColumn dtypes in df_ward:')
df_ward.dtypes

In [ ]:
# Find columns that are already datetime in df_ward
date_cols_existing = [c for c in df_ward.columns if pd.api.types.is_datetime64_any_dtype(df_ward[c])]
print(f"\nColumns with datetime dtype: {len(date_cols_existing)} found")
print(date_cols_existing)

In [ ]:
# Try to parse columns with 'date' in the name (in case they're strings)
date_cols_by_name = [c for c in df_ward.columns if 'date' in c.lower() and c not in date_cols_existing]
print('Columns with "date" in name :', date_cols_by_name)

In [ ]:
# Parse date columns in-place using format='mixed'
for c in date_cols_by_name:
    df_ward[c] = pd.to_datetime(df_ward[c], format='mixed', errors='coerce')
    parsed_count = df_ward[c].notna().sum()
    print(f"  {c}: {parsed_count} dates parsed successfully")

# Now all date columns are datetime objects
all_date_cols = date_cols_existing + date_cols_by_name
print(f"\nTotal date columns available: {len(all_date_cols)}")

In [ ]:
print('All usable date columns:')
all_date_cols


In [ ]:
df_ward.dtypes

In [ ]:
df_ward['HEARING DATE']

In [ ]:
# Create CityDelay using the HEARING DATE and VIOLATION DATE columns
hearing_col = 'HEARING DATE'
violation_col = 'VIOLATION DATE'

if hearing_col in df_ward.columns and violation_col in df_ward.columns:
    df_ward['CityDelay'] = (df_ward[hearing_col] - df_ward[violation_col]).dt.days
    print(f'CityDelay column created (in days) using:')
    print(f'  HearingDate: {hearing_col}')
    print(f'  ViolationDate: {violation_col}')
    print(f'\nCityDelay statistics:')
    print(df_ward['CityDelay'].describe())
else:
    print(f'Error: Could not find columns {hearing_col} or {violation_col}')

# Save a small sample of processed dataframe (from df_ward)
df_ward.head(100).to_csv(out_dir / 'data_sample_head100.csv', index=False)
print(f'\nSaved sample head to outputs/data_sample_head100.csv')

In [ ]:
# Birthday analysis: ask user for day and month
print('\nBirthday analysis.')

birthday_day = 5
birthday_month = 10

if birthday_day and birthday_month and violation_col in df_ward.columns:
    # Count violations on birthday (any year)
    is_bday = (df_ward[violation_col].dt.day == birthday_day) & (df_ward[violation_col].dt.month == birthday_month)
    bday_df = df_ward[is_bday]
    print(f"Violations on birthday ({birthday_day}/{birthday_month}): {len(bday_df)}")
    if len(bday_df) > 0 and 'CASE DISPOSITION' in bday_df.columns:
        print('Most common Case Disposition on that day:')
        print(bday_df['CASE DISPOSITION'].value_counts().head())

## Complete till q - 5

In [ ]:
print("\n" + "=" * 90)
print("GROUPBY ANALYSIS: Unique Values by Ward")
print("=" * 90)

In [ ]:
# Group by WARD and show as a formatted table
# size will return a series so converted it to DF with reset_index with two columns
ward_counts = df.groupby('WARD').size().reset_index(name='Total Size')
ward_counts


In [ ]:
ward_counts.columns = ['Ward Number', 'Total Size']

# Display as table
print("\n### Violations by Ward ###")
print(ward_counts.to_string(index=False))

# Also display as pandas DataFrame for nice HTML rendering
ward_counts

## Should Ward be considered a numeric or categorical variable? Why?

Based on the values analyzed in the cell above I can inferit is a cateforical variable

- Discrete Labels — Ward values are 1, 2, 3... 43, 44, 50
- No Magnitude Meaning — Ward 20 isn't "twice as bad" as Ward 10
- No Natural Order — Can't compute mean ward = 14.5 or median ward meaningfully
- Identical Treatment — Each ward should be treated as a separate geographic area
- Operations Make No Sense — Adding/subtracting ward numbers produces garbage

In [ ]:
# Group by WARD 
# Convert WARD to numeric
ward_numeric = pd.to_numeric(df['WARD'], errors='coerce')

# Select key columns to analyze
key_columns = ['STREET NAME', 'VIOLATION CODE', 'CASE DISPOSITION', 'VIOLATION DESCRIPTION']

# Create a summary table
summary_data = []
for ward in sorted(ward_numeric.dropna().unique()):
    ward_df = df[ward_numeric == ward]
    row = {'WARD': int(ward), 'Total Records': len(ward_df)}
    
    for col in key_columns:
        if col in df.columns:
            unique_count = ward_df[col].nunique()
            row[f'Unique {col}'] = unique_count
    
    summary_data.append(row)

# Create DataFrame
summary_table = pd.DataFrame(summary_data)

print("\n### Summary Table: Unique Counts by Ward ###")
print(summary_table.to_string(index=False))

# Save to CSV
summary_table.to_csv(out_dir / 'ward_summary_unique_counts.csv', index=False)
print(f"\n✓ Saved to outputs/ward_summary_unique_counts.csv")

# Show detailed view for chosen ward
print(f"\n" + "=" * 90)
print(f"DETAILED BREAKDOWN FOR WARD {chosen_ward}")
print("=" * 90)

chosen_ward_data = df[ward_numeric == chosen_ward]

for col in key_columns:
    if col in df.columns:
        unique_vals = chosen_ward_data[col].nunique()
        print(f"\n{col}: {unique_vals} unique values")
        print(f"  Top 5 most common:")
        top_vals = chosen_ward_data[col].value_counts().head()
        for val, count in top_vals.items():
            print(f"    • {val}: {count} records")

In [ ]:
# Correlation between Imposed Fine and Admin Costs
print("\n" + "=" * 90)
print("QUESTION 2: Correlation between Imposed Fine and Admin Costs")
print("=" * 90)

The **Pearson correlation coefficient** (often denoted as `r` for a sample or `p` for a population), is a statistical measure that quantifies the linear correlation between two sets of continuous data, `x` and `y`

In [ ]:
# Define the fine and admin cost columns directly
fine_col = 'IMPOSED FINE'
admin_col = 'ADMIN COSTS'

# Convert to numeric
df_ward[fine_col] = pd.to_numeric(df_ward[fine_col], errors='coerce')
df_ward[admin_col] = pd.to_numeric(df_ward[admin_col], errors='coerce')

# Remove NaN values for correlation
valid_data = df_ward[[fine_col, admin_col]].dropna()

if len(valid_data) > 0:
    print(f"\nColumns analyzed:")
    print(f"  • Fine column: {fine_col}")
    print(f"  • Admin cost column: {admin_col}")
    print(f"  • Valid records (non-null both): {len(valid_data)}")
    
    # Compute correlation
    correlation = valid_data[fine_col].corr(valid_data[admin_col])
    print(f"Pearson correlation coefficient: {correlation:.4f}")
    
    # Interpret correlation
    if abs(correlation) < 0.3:
        strength = "WEAK"
    elif abs(correlation) < 0.7:
        strength = "MODERATE"
    else:
        strength = "STRONG"
    
    direction = "positive" if correlation > 0 else "negative"
    
    print(f"Strength: {strength}")
    print(f"Direction: {direction}")
    
    

In [ ]:
print(f"""
• Correlation = {correlation:.4f}: {strength} {direction} relationship
• Meaning: {'As imposed fines increase, admin costs tend to increase' if correlation > 0 else 'As imposed fines increase, admin costs tend to decrease'}
• Strength insight: 
    - If 'WEAK': Fine amount and admin costs are largely independent
    - If 'MODERATE': There's some relationship but other factors matter too
    - If 'STRONG': Fine and admin costs move together closely

The {strength.lower()} {direction} correlation (r={correlation:.4f}) suggests that:
    → Imposed fines and admin costs are {'somewhat' if abs(correlation) < 0.7 else ''} related
    → Different violations likely have different penalty structures
    → City may assign admin costs based on factors other than fine amount
    """)
    

In [ ]:
most_common_street_type = df_ward['STREET TYPE'].value_counts()
print(f" 2. Most common street type : {most_common_street_type.index[0]}")

## No I do not like in the same street type.

In [ ]:
# Count unique Violation Description and Violation Code values in Ward 20
print(f"UNIQUE VIOLATION VALUES IN WARD {chosen_ward}")

unique_violation_codes = df_ward['VIOLATION CODE'].nunique()
unique_violation_descriptions = df_ward['VIOLATION DESCRIPTION'].nunique()

print(f"\nUnique VIOLATION CODE values: {unique_violation_codes}")
print(f"Unique VIOLATION DESCRIPTION values: {unique_violation_descriptions}")

print(f"\nTotal records in Ward {chosen_ward}: {len(df_ward)}")
print(f"Average violations per unique code: {len(df_ward) / unique_violation_codes:.2f}")

In [ ]:
# Show relationship between codes and descriptions
print(f"\n### Top 5 Most Common Violation Codes ###")
top_codes = df_ward['VIOLATION CODE'].value_counts().head(5)
for code, count in top_codes.items():
    desc = df_ward[df_ward['VIOLATION CODE'] == code]['VIOLATION DESCRIPTION'].iloc[0]
    print(f"{code}: {count} occurrences")
    print(f"   Description: {desc}\n")

## Item 7: Imposed fines

In [ ]:
# Average Imposed Fine by Year (from HEARING DATE) in Ward 20
print("AVERAGE IMPOSED FINE BY YEAR (Ward 20)")

# Extract year from HEARING DATE and convert to integer
df_ward['Hearing_Year'] = df_ward['HEARING DATE'].dt.year.astype('Int64')  # Int64 handles NaN values
df_ward['Hearing_Year']

In [ ]:
# Convert IMPOSED FINE to numeric
df_ward['IMPOSED FINE'] = pd.to_numeric(df_ward['IMPOSED FINE'], errors='coerce')
df_ward['IMPOSED FINE'].sort_values(ascending=False).head(5)

In [ ]:
# Group by year and calculate average fine
avg_fine_by_year = df_ward.groupby('Hearing_Year')['IMPOSED FINE'].agg([
    ('Count', 'size'),
    ('Average Fine', 'mean'),
    ('Median Fine', 'median'),
    ('Min Fine', 'min'),
    ('Max Fine', 'max'),
    ('Std Dev', 'std')
]).reset_index()

avg_fine_by_year

In [ ]:
# Rename the Hearing_Year column to Year
avg_fine_by_year = avg_fine_by_year.rename(columns={'Hearing_Year': 'Year'})

print("\n Average Imposed Fine Statistics by Year \n")

avg_fine_by_year

In [ ]:
# Save to CSV
avg_fine_by_year.to_csv(out_dir / 'average_fine_by_year.csv', index=False)

print(f"\n Saved to outputs/average_fine_by_year.csv")

In [ ]:
# Show trend summary as a table
print(f"\n YEAR-OVER-YEAR TREND \n")

# Create a summary dataframe for display
trend_summary = []
years = sorted([y for y in df_ward['Hearing_Year'].unique() if pd.notna(y)])
for year in years:
    avg = df_ward[df_ward['Hearing_Year'] == year]['IMPOSED FINE'].mean()
    count = len(df_ward[df_ward['Hearing_Year'] == year])
    trend_summary.append({
        'Year': int(year),
        'Count': count,
        'Average Fine': f"${avg:,.2f}"
    })

trend_df = pd.DataFrame(trend_summary)
print(trend_df.to_string(index=False))

In [ ]:
# Create line graph for year-over-year trend
plt.figure(figsize=(12, 6))

# Plot average fine by year
plt.plot(avg_fine_by_year['Year'], avg_fine_by_year['Average Fine'], 
         marker='o', linewidth=2, markersize=8, color='#1f77b4', label='Average Fine')

# Add median fine line for comparison
plt.plot(avg_fine_by_year['Year'], avg_fine_by_year['Median Fine'], 
         marker='s', linewidth=2, markersize=8, color='#ff7f0e', linestyle='--', label='Median Fine')

plt.xlabel('Year', fontsize=12, fontweight='bold')
plt.ylabel('Fine Amount ($)', fontsize=12, fontweight='bold')
plt.title(f'Ward {chosen_ward}: Average & Median Imposed Fines by Year', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.legend(fontsize=10)
plt.xticks(avg_fine_by_year['Year'])
plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:,.0f}'))

plt.tight_layout()
plt.savefig(out_dir / 'imposed_fines_by_year.png', dpi=300, bbox_inches='tight')
print("\n✓ Line graph saved to outputs/imposed_fines_by_year.png")
plt.show()

### Why are 2024 Imposed Fines Lower?

In [ ]:
# Get 2024 data
df_2024 = df_ward[df_ward['Hearing_Year'] == 2024]
df_prev_years = df_ward[df_ward['Hearing_Year'] < 2024]

In [ ]:
print(f"2024 records: {len(df_2024)}")
print(f"Previous years records: {len(df_prev_years)}")

print(f"\n2024 IMPOSED FINE - Non-null count: {df_2024['IMPOSED FINE'].notna().sum()}")
print(f"2024 IMPOSED FINE - Null/Missing count: {df_2024['IMPOSED FINE'].isna().sum()}")
print(f"Percentage of 2024 with valid fines: {100 * df_2024['IMPOSED FINE'].notna().sum() / len(df_2024):.1f}%")


In [ ]:
print(f"\nPrevious years IMPOSED FINE - Non-null count: {df_prev_years['IMPOSED FINE'].notna().sum()}")
print(f"Previous years IMPOSED FINE - Null/Missing count: {df_prev_years['IMPOSED FINE'].isna().sum()}")
print(f"Percentage with valid fines: {100 * df_prev_years['IMPOSED FINE'].notna().sum() / len(df_prev_years):.1f}%")

In [ ]:
# Check if 2024 data is partial year
min_date_2024 = df_2024['HEARING DATE'].min()
max_date_2024 = df_2024['HEARING DATE'].max()
print(f"2024 Hearing Date Range: {min_date_2024.date()} to {max_date_2024.date()}")

# Compare months covered
months_2024 = df_2024['HEARING DATE'].dt.month.unique()
months_2024_sorted = sorted([int(m) for m in months_2024])

print(f"Months represented in 2024: {months_2024_sorted}")
print(f"Number of unique months in 2024: {len(months_2024_sorted)}")

In [ ]:
print(f"\nVIOLATION DISTRIBUTION ")
# Check if 2024 has different violation types
print(f"\nTop 5 Violation Codes in 2024:")
print(df_2024['VIOLATION CODE'].value_counts().head())

In [ ]:
print(f"\nTop 5 Violation Codes in Previous Years:")
print(df_prev_years['VIOLATION CODE'].value_counts().head())

In [ ]:
# Check average fine by violation code
print(f"\nAVERAGE FINE BY VIOLATION CODE")
code_2024 = df_2024.groupby('VIOLATION CODE')['IMPOSED FINE'].mean().sort_values(ascending=False)
code_prev = df_prev_years.groupby('VIOLATION CODE')['IMPOSED FINE'].mean().sort_values(ascending=False)

print(f"\nTop 5 highest average fines in 2024:")
print(code_2024.head())

print(f"\nTop 5 highest average fines in previous years:")
print(code_prev.head())

In [ ]:
print(f"\nCONCLUSION")
print(f"""
POSSIBLE REASONS FOR LOWER 2024 AVERAGE FINES:

1. INCOMPLETE DATA FOR 2024:
   - 2024 data covers: {len(months_2024)} months ({sorted(months_2024)})
   - If 2024 is partial-year data, it may not represent full year patterns
   
""")

In [ ]:
# Remove columns 'ID' and 'DOCKET NUMBER'
cols_to_drop = ['ID', 'DOCKET NUMBER']

print("Before dropping columns:")
print(f"df_ward shape: {df_ward.shape}")
print(f"Columns: {list(df_ward.columns)}")

# Drop columns
df_ward = df_ward.drop(columns=cols_to_drop, errors='ignore')

print(f"\nAfter dropping columns:")
print(f"df_ward shape: {df_ward.shape}")
print(f"Columns: {list(df_ward.columns)}")

In [ ]:
# Create SEASON column from VIOLATION DATE
# Extract quarter from VIOLATION DATE and map to season names

# Define quarter to season mapping
quarter_to_season = {
    1: 'Winter',
    2: 'Spring',
    3: 'Summer',
    4: 'Fall'
}

In [ ]:
# Extract quarter from VIOLATION DATE and map to season
df_ward['SEASON'] = df_ward['VIOLATION DATE'].dt.quarter.map(quarter_to_season)

df_ward.head(5)

In [ ]:
print("First 10 rows of VIOLATION DATE and SEASON:")
print(df_ward[['VIOLATION DATE', 'SEASON']].head(10))

In [ ]:
print("\n\nSeason distribution in Ward 20:")
print(df_ward['SEASON'].value_counts().sort_index())

In [ ]:
# Create a bar plot of violations by season
plt.figure(figsize=(10, 6))

season_order = ['Winter', 'Spring', 'Summer', 'Fall']
season_counts = df_ward['SEASON'].value_counts().sort_index() # Sort alphabetically
season_counts = season_counts.reindex(season_order)

plt.bar(season_counts.index, season_counts.values, color='steelblue', edgecolor='black', linewidth=1.2)

plt.xlabel('Season', fontsize=12, fontweight='bold')
plt.ylabel('Number of Violations', fontsize=12, fontweight='bold')
plt.title(f'Building Violations by Season in Ward {chosen_ward}', fontsize=14, fontweight='bold')
plt.grid(True, axis='y', alpha=0.3)

# Add count labels on top of bars
for i, (season, count) in enumerate(zip(season_counts.index, season_counts.values)):
  plt.text(i, count + 50, str(int(count)), ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.savefig(out_dir / 'violations_by_season.png', dpi=300, bbox_inches='tight')
print("\n Bar plot saved to outputs/violations_by_season.png")
plt.show()

### Pattern

From the graph above it is evident that winter has the highest violations with a declining pattern

#### Why it looks this way?

##### Winter & lower activity 

- Winter has more violations likely due to increased indoor renovation/repair work during cold months
- Building owners may accelerate projects before winter or do emergency repairs

##### Building Maintenance Cycles

- Spring/Summer violations remain high because outdoor repairs continue
- However, these months show a slight decrease from winter

In [ ]:
# Filter to keep only rows with the 5 most common CASE DISPOSITION values
df_ward['CASE DISPOSITION'].value_counts().head(5)

In [ ]:
top_5_dispositions = df_ward['CASE DISPOSITION'].value_counts().head(5).index.tolist()

print("Top 5 most common CASE DISPOSITION values:")
top_5_dispositions


In [ ]:
# Filter df_ward to keep only these dispositions
df_ward_filtered = df_ward[df_ward['CASE DISPOSITION'].isin(top_5_dispositions)].copy()

print(f"\nOriginal df_ward shape: {df_ward.shape}")
print(f"Filtered df_ward shape: {df_ward_filtered.shape}")
print(f"Rows removed: {df_ward.shape[0] - df_ward_filtered.shape[0]}")

print(f"\nCASE DISPOSITION distribution in filtered dataset:")
print(df_ward_filtered['CASE DISPOSITION'].value_counts())

# 1. Count rows in df_ward_filtered
print(f"Number of rows in df_ward_filtered: {len(df_ward_filtered)}")

In [ ]:
# Create histograms for CityDelay by Case Disposition

# Define a list of 5 distinct colors (one for each case disposition type)
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']

# Get the list of top 5 dispositions we filtered above
dispositions = top_5_dispositions

# Create a figure with multiple subplots stacked vertically (5 rows, 1 column)
# Each subplot will show one histogram for one case disposition type
fig, axes = plt.subplots(5, 1, figsize=(12, 4 * 5))

# Loop through each disposition and create a histogram for it
for idx, disposition in enumerate(dispositions):
    # Filter data: get all CityDelay values for this specific case disposition
    # .dropna() removes any missing values (null/NaN entries)
    disp_data = df_ward_filtered[df_ward_filtered['CASE DISPOSITION'] == disposition]['CityDelay'].dropna()
    
    # Handle outliers by using the 99th percentile
    limit = disp_data.quantile(0.99)

    # Filter to keep only data points at or below the 99th percentile
    plot_data = disp_data[disp_data <= limit]
    
    # Create a histogram on the current subplot (axes[idx])
    # bins=30 means divide the data into 30 equal-width buckets
    axes[idx].hist(plot_data, bins=30, color=colors[idx], edgecolor='black', linewidth=1.2, alpha=0.8)
    
    # Label the x-axis (horizontal axis) to show it represents days of delay
    axes[idx].set_xlabel('City Delay (days)', fontsize=11, fontweight='bold')
    
    # Label the y-axis (vertical axis) to show it represents frequency/count
    axes[idx].set_ylabel('Frequency', fontsize=11, fontweight='bold')
    
    # Set the title for this histogram, showing the disposition name and total sample size (n)
    axes[idx].set_title(f'{disposition} (n={len(disp_data)})', fontsize=12, fontweight='bold')
    
    # Add a light grid to the y-axis only to make it easier to read values
    axes[idx].grid(True, axis='y', alpha=0.3)

# Automatically adjust spacing between subplots to prevent overlap
plt.tight_layout()

# Save the entire figure as a PNG image to the outputs folder
# dpi=300 means high resolution (300 dots per inch)
plt.savefig(out_dir / 'citydelay_by_disposition_histograms.png', dpi=300, bbox_inches='tight')

# Print a confirmation message to show the file was saved successfully
print(" Histograms saved to outputs/citydelay_by_disposition_histograms.png")

# Display the figure in the notebook
plt.show()

**What the plot shows:**

The histograms display the processing time (in days) from violation date to hearing date for each of the 5 most common case dispositions in Ward 20. This reveals how quickly the city handles different types of building violation cases.

**Key findings that stand out:**

1. **Liable Cases are MODERATELY DELAYED (~200-300 days peak):**
   - The histogram shows a broad, relatively normal distribution centered around 250-300 days
   - The distribution is more spread out than Default cases, suggesting more variability
   - *Why:* Even though the defendant is found liable, the process involves evidence review, hearings, and official rulings, which takes substantial time

2. **Non-Suit Cases have HIGH VARIABILITY (200-400+ days, very dispersed):**
   - The histogram shows a wide, dispersed distribution with a long right tail extending past 800+ days
   - This is the most variable disposition with some cases taking exceptionally long
   - *Why:* "Non-suit" cases often involve dismissals, procedure issues, or settlements that require back-and-forth negotiation. Some cases get stuck or require multiple filings, explaining the extreme range

3. **Default Cases are FASTEST & MOST CONCENTRATED (~100-200 days):**
   - The histogram shows a sharp, tight peak with the shortest range of delays
   - Most cases cluster tightly between 50-250 days
   - *Why:* When defendants don't appear, the ruling is automatic and quick—no need for evidence review or trial proceedings

**Overall** Cases requiring judicial review (Liable, Non-Suit, Continuance) take significantly longer than administrative rulings (Default). The more procedurally complex a case, the more variable the processing time.


In [ ]:
# Create a bar plot showing average imposed fine by case disposition
# Calculate average fine for each disposition in the filtered dataset
avg_fine_by_disposition = df_ward_filtered.groupby('CASE DISPOSITION')['IMPOSED FINE'].mean().sort_values(ascending=False)
avg_fine_by_disposition

In [ ]:

# Create the bar plot using the colors defined above
plt.figure(figsize=(12, 6))
bars = plt.bar(range(len(avg_fine_by_disposition)), avg_fine_by_disposition.values, 
               color=colors, edgecolor='black', linewidth=1.2, alpha=0.85)

# Set x-axis labels to disposition names
plt.xticks(range(len(avg_fine_by_disposition)), avg_fine_by_disposition.index, fontsize=11, fontweight='bold')

# Label axes
plt.xlabel('Case Disposition', fontsize=12, fontweight='bold')
plt.ylabel('Average Imposed Fine ($)', fontsize=12, fontweight='bold')
plt.title(f'Ward {chosen_ward}: Average Imposed Fine by Case Disposition', fontsize=14, fontweight='bold')

# Format y-axis as currency
plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:,.0f}'))

# Add grid for easier reading
plt.grid(True, axis='y', alpha=0.3)

# Add value labels on top of each bar
for i, (bar, value) in enumerate(zip(bars, avg_fine_by_disposition.values)):
    plt.text(bar.get_x() + bar.get_width()/2, value + 50, f'${value:,.0f}', 
             ha='center', va='bottom', fontweight='bold', fontsize=10)

plt.tight_layout()
plt.savefig(out_dir / 'avg_fine_by_disposition_barplot.png', dpi=300, bbox_inches='tight')
print(" Bar plot saved to outputs/avg_fine_by_disposition_barplot.png")
plt.show()

In [ ]:
# Print summary statistics
print("\nAverage Imposed Fine by Case Disposition (sorted from highest to lowest):")
print(avg_fine_by_disposition.to_string())


**What the plot shows:**

The bar chart above displays the average imposed fine (penalty amount) for each of the 5 most common case dispositions in Ward 20 (we selected), sorted from highest to lowest. 

This reveals how the City's penalty structure varies significantly based on the legal outcome of each case.

**Key observations:**

1. **Default Cases ($4,950 - HIGHEST FINES):**
   - Default cases receive dramatically higher average fines than all other dispositions
   - The gap between Default ($4,950) and Liable ($958) is more than 5 times.
   - *Why:* When defendants fail to appear or respond, the City likely pursues the maximum penalty as enforcement. Default judgments may include automatic penalty assessments or higher starting fines to incentivize compliance and future appearances.

2. **Liable Cases ($958 - MODERATE):**
   - Second highest average fine, but still much lower than Default
   - *Why:* When the City proves the violation ("Liable"), penalties are assessed based on violation severity and evidence presented. These are proportional to the specific building code violation rather than punitive for non-appearance.

3. **Continuance Cases ($39 - VERY LOW):**
   - Cases postponed to future dates have minimal average fines
   - *Why:* Continuances are temporary postponements where no ruling has yet been made. Cases likely haven't reached adjudication, so no final penalty has been assessed. The small amount may represent procedural costs or interim penalties.

4. **Non-Suit Cases ($32 - VERY LOW):**
   - When plaintiffs withdraw cases, virtually no fines are imposed
   - *Why:* Non-suits represent dismissed cases—no violation was proven. The City has no grounds to impose penalties on dismissed claims.

5. **Not Liable Cases ($3 - LOWEST):**
   - Defendants found "Not Liable" face almost no financial penalty
   - *Why:* When the defendant is cleared of charges, the City cannot impose fines. The minimal amount may be rounding or administrative fees.

**Overall pattern & explanation:**

The data reveals a **clear legal principle**: penalties are tied to culpability and case outcome:

- **Failure to respond/appear (Default)** = Highest penalties (enforcement/deterrent)
- **Guilt proven (Liable)** = Moderate penalties (proportional to violation)  
- **No fault/dismissal (Not Liable, Non-Suit, Continuance)** = Minimal/no penalties (innocent until proven guilty)

This structure suggests the City uses the fine system not just for revenue, but as a compliance tool—penalizing non-engagement (Defaults) severely while reserving moderate penalties for proven violations.


In [ ]:
# Create a stacked histogram showing Admin Costs distribution by Case Disposition
# Clean the data: remove NaN values from ADMIN COSTS
admin_data = df_ward_filtered[['ADMIN COSTS', 'CASE DISPOSITION']].dropna()

# Convert ADMIN COSTS to numeric
admin_data['ADMIN COSTS'] = pd.to_numeric(admin_data['ADMIN COSTS'], errors='coerce')
admin_data = admin_data.dropna()

admin_data

In [ ]:
# Create the figure
plt.figure(figsize=(14, 7))

# Get unique dispositions
dispositions_list = admin_data['CASE DISPOSITION'].unique()

# Create a stacked histogram with no gaps between bins
# Use rwidth=1.0 to ensure no gaps between bins
for i, disposition in enumerate(dispositions_list):
    # Filter data for this disposition
    disp_costs = admin_data[admin_data['CASE DISPOSITION'] == disposition]['ADMIN COSTS']
    
    # Plot histogram for this disposition with alpha for transparency (for stacking effect)
    plt.hist(disp_costs, bins=50, alpha=0.6, label=disposition, color=colors[i], 
             edgecolor='black', linewidth=0.5)

# Label axes and title
plt.xlabel('Admin Costs ($)', fontsize=12, fontweight='bold')
plt.ylabel('Frequency', fontsize=12, fontweight='bold')
plt.title(f'Ward {chosen_ward}: Distribution of Admin Costs by Case Disposition', 
          fontsize=14, fontweight='bold')

# Add legend
plt.legend(fontsize=10, loc='upper right')

# Format x-axis as currency
plt.gca().xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:,.0f}'))

# Add grid
plt.grid(True, axis='y', alpha=0.3)

# Save and display
plt.tight_layout()
plt.savefig(out_dir / 'admin_costs_by_disposition_histogram.png', dpi=300, bbox_inches='tight')
print(" Histogram saved to outputs/admin_costs_by_disposition_histogram.png")
plt.show()



In [ ]:
# Print summary statistics
print("\nAdmin Costs Summary by Case Disposition:")
print(admin_data.groupby('CASE DISPOSITION')['ADMIN COSTS'].describe())

**What the plot shows:**

The histogram displays the distribution of administrative costs (processing fees) across the five most common case dispositions in Ward 20, with each disposition shown as a transparent overlay. 

Admin costs represent the operational expenses the City incurs in handling each building violation case.

**Key observations:**

1. **Non-Suit Cases Dominate at $0 (MASSIVE SPIKE):**

   - The tallest peak appears at $0 admin costs, dominated by Non-Suit cases (orange)
   - Nearly 10,000 Non-Suit cases have zero associated admin costs
   - *Why:* When a case is dismissed (non-suited), minimal administrative processing occurs. The City has already closed the file, so there are no ongoing costs to track or bill

2. **Liable Cases Show Concentrated Distribution ($50-75 range - NOTABLE CLUSTER):**
 
   - Liable cases (green) form a secondary peak around $50-75
   - This represents roughly 8,500 cases with moderate administrative costs
   - *Why:* Liable cases require full adjudication—evidence review, hearings, record-keeping, and official determinations. These legal proceedings generate consistent administrative overhead

3. **Default Cases Spread Across Mid-Range ($50-100+):**
 
   - Default cases (blue) show a broader distribution centered around $50-100
   - Approximately 4,500 cases distributed across this range
   - *Why:* Defaults still require administrative follow-up for non-response, collection attempts, and case closure documentation

4. **Not Liable & Continuance Cases Show Minimal Spread:**
 
   - Not Liable cases (red) and Continuance cases (purple) are sparsely distributed
   - These represent fewer total cases and cluster near lower cost ranges
   - *Why:* Not Liable cases are quickly dismissed (minimal paperwork). Continuance cases are merely postponed, so full administrative processing hasn't occurred yet

**Overall pattern & explanation:**

The histogram reveals a **stark cost-to-outcome relationship**:

- **Dismissed cases (Non-Suit)** = Zero admin costs (no resources expended)
- **Adjudicated cases (Liable, Default)** = Moderate, consistent costs ($50-100)
- **Incomplete/Dismissed cases (Not Liable, Continuance)** = Minimal costs

This suggests administrative costs directly reflect case complexity and judicial effort.

In [ ]:
# Display all unique VIOLATION DESCRIPTION values and their counts
violation_desc_counts = df_ward_filtered['VIOLATION DESCRIPTION'].value_counts()

print(f"\nTotal unique violation descriptions: {len(violation_desc_counts)}")
print("\nAll violation descriptions with counts:")
print(violation_desc_counts.to_string())


In [ ]:
# Get the top 5 most common VIOLATION DESCRIPTION values
top_5_descriptions = df_ward_filtered['VIOLATION DESCRIPTION'].value_counts().head(5)

print("\nTop 5 violation descriptions:")
for idx, (description, count) in enumerate(top_5_descriptions.items(), 1):
    print(f"{idx}. {description}: {count} cases")

print(f"\nTotal cases represented by top 5 descriptions: {top_5_descriptions.sum()}")
print(f"Percentage of dataset: {100 * top_5_descriptions.sum() / len(df_ward_filtered):.2f}%")


In [ ]:
# Filter to keep only rows with the 5 most common VIOLATION DESCRIPTION values
top_5_violation_descriptions = df_ward_filtered['VIOLATION DESCRIPTION'].value_counts().head(5).index.tolist()

df_ward_filtered_by_desc = df_ward_filtered[df_ward_filtered['VIOLATION DESCRIPTION'].isin(top_5_violation_descriptions)].copy()

print(f"\nOriginal df_ward_filtered shape: {df_ward_filtered.shape}")
print(f"Filtered by violation description shape: {df_ward_filtered_by_desc.shape}")
print(f"Rows removed: {df_ward_filtered.shape[0] - df_ward_filtered_by_desc.shape[0]}")


In [ ]:
print(f"\nVIOLATION DESCRIPTION distribution in newly filtered dataset:")
print(df_ward_filtered_by_desc['VIOLATION DESCRIPTION'].value_counts())

print(f"\nNumber of rows in df_ward_filtered_by_desc: {len(df_ward_filtered_by_desc)}")


In [ ]:
# Create a mapping from full violation descriptions to short labels
violation_desc_mapping = {
    'Arrange for inspection of premises. (13-12-100)': 'Inspection Access',
    'Repair or replace defective or missing members of porch system. (13-196-570)': 'Porch System',
    'Repair exterior wall. (13-196-010, 13-196-530 B)': 'Exterior Wall',
    'Post name, address, and telephone of owner, owner\'s agent for managing, controlling or collecting rents, and any other person managing or controlling building conspicuously where accessible or visible to public way. (13-12-030)': 'Owner Info',
    'Replace broken, missing or defective window panes. (13-196-550 A)': 'Window Panes'
}


In [ ]:
print("Violation Description Shortening Mapping:")

for full, short in violation_desc_mapping.items():
    print(f"\nORIGINAL: {full}")
    print(f"SHORT:    {short}")

In [ ]:
# Apply the mapping to create a new column with shortened descriptions
df_ward_filtered_by_desc['VIOLATION_SHORT'] = df_ward_filtered_by_desc['VIOLATION DESCRIPTION'].map(violation_desc_mapping)

print("\nDataset with shortened violation descriptions:")
print(f"Shape: {df_ward_filtered_by_desc.shape}")


In [ ]:
print(f"\nFirst 10 rows (showing original and shortened descriptions):")
print(df_ward_filtered_by_desc[['VIOLATION DESCRIPTION', 'VIOLATION_SHORT']].head(10))

In [ ]:
print(f"\nDistribution of shortened violation descriptions:")
print(df_ward_filtered_by_desc['VIOLATION_SHORT'].value_counts())

In [ ]:
# Calculate mean imposed fine for each violation description
mean_fine_by_violation = df_ward_filtered_by_desc.groupby('VIOLATION_SHORT')['IMPOSED FINE'].mean().sort_values(ascending=False)

print("Mean Imposed Fine by Violation Type (sorted highest to lowest):")
print(mean_fine_by_violation.to_string())

In [ ]:
# Create a bar plot showing mean imposed fine by violation type
plt.figure(figsize=(12, 6))

# Define a custom color palette
violation_colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A', '#98D8C8']

bars = plt.bar(range(len(mean_fine_by_violation)), mean_fine_by_violation.values, 
               color=violation_colors, edgecolor='black', linewidth=1.2, alpha=0.85)

# Set x-axis labels to violation short names
plt.xticks(range(len(mean_fine_by_violation)), mean_fine_by_violation.index, fontsize=11, fontweight='bold')

# Label axes
plt.xlabel('Violation Type', fontsize=12, fontweight='bold')
plt.ylabel('Mean Imposed Fine ($)', fontsize=12, fontweight='bold')
plt.title(f'Ward {chosen_ward}: Mean Imposed Fine by Violation Type', fontsize=14, fontweight='bold')

# Format y-axis as currency
plt.gca().yaxis.set_major_formatter(plt.a(lambda x, p: f'${x:,.0f}'))

# Add grid for easier reading
plt.grid(True, axis='y', alpha=0.3)

# Add value labels on top of each bar
for i, (bar, value) in enumerate(zip(bars, mean_fine_by_violation.values)):
    plt.text(bar.get_x() + bar.get_width()/2, value + 20, f'${value:,.0f}', 
             ha='center', va='bottom', fontweight='bold', fontsize=10)

plt.tight_layout()
plt.savefig(out_dir / 'mean_fine_by_violation_type.png', dpi=300, bbox_inches='tight')
print("\n Bar plot saved to outputs/mean_fine_by_violation_type.png")
plt.show()

**What this plot shows:**

The bar chart displays the average penalty amount imposed for each of the 5 most common building violation types in Ward 20, sorted from highest to lowest fines. This reveals which violation categories the City penalizes most heavily.

**Key findings (sorted by severity of penalties):**

1. **Window Panes ($1,539 - HIGHEST FINE):**
   - Nearly double the fine for Inspection Access violations
   - *Why:* Windows are critical for building safety, security, and weatherproofing. Broken/missing window panes expose interiors to weather damage, pest infiltration, and security risks. The high fine incentivizes rapid replacement to prevent escalating property damage.

2. **Exterior Wall ($1,205 - SECOND HIGHEST):**
   - Structural integrity is paramount—exterior walls protect the entire building
   - *Why:* Wall deterioration compromises structural stability and exposes the building to water damage, mold, and potential collapse. The city prioritizes these repairs through substantial fines.

3. **Porch System ($1,107 - MODERATE-HIGH):**
   - Porches are structural extensions that can fail and injure occupants if neglected
   - *Why:* Failed porch systems pose immediate safety hazards (collapse risk). The moderate-high fine reflects the danger but is lower than walls because they're secondary structures.

4. **Owner Info ($1,041 - MODERATE):**
   - Administrative requirement to post owner contact information
   - *Why:* This is primarily a compliance/administrative issue—it doesn't directly threaten building safety but allows tenants/city to contact responsible parties. Fines are moderate to encourage compliance without excessive punishment.

5. **Inspection Access ($854 - LOWEST FINE):**
   - Administrative requirement to allow city inspectors access
   - *Why:* This is the least "structural"—it's about process and cooperation. The lower fine reflects that violation of inspection access is more about obstructing oversight than causing direct physical harm.

**Overall pattern & explanation:**

The fines follow a clear **safety hierarchy**:

| Rank | Violation Type | Fine | Category |
|---|---|---|---|
| 1 | Window Panes | $1,539 | **Critical Safety** (weatherproofing, security) |
| 2 | Exterior Wall | $1,205 | **Structural Integrity** (building stability) |
| 3 | Porch System | $1,107 | **Secondary Structure** (occupant safety) |
| 4 | Owner Info | $1,041 | **Administrative** (compliance, accountability) |
| 5 | Inspection Access | $854 | **Procedural** (oversight/cooperation) |

**Conclusion:**

The City's fine structure is **risk-based**: violations threatening immediate physical safety and structural integrity receive substantially higher penalties ($1,200-$1,500 range) than administrative compliance issues ($850-$1,000 range). 

This strategy incentivizes building owners to focus on life-safety issues first, then handle administrative requirements second.
